<a href="https://colab.research.google.com/github/ZainAli24/Quater_04/blob/main/class2_OpenAI_Agents_SDK_Quiz_Prepration(Revision).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.6/734.6 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [4]:
from agents import Agent,Runner,AsyncOpenAI,OpenAIChatCompletionsModel
from agents.run import RunConfig

external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)

model1 = OpenAIChatCompletionsModel(
    model="gemini-1.5-flash",
    openai_client=external_client
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True
)


In [5]:
agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model=model1
)


result = await Runner.run(
    agent,
    "Add 3 in 67 ",
    run_config=config,
)

print(result.new_items)

[MessageOutputItem(agent=Agent(name='Assistant', instructions='You are a helpful assistant.', prompt=None, handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x7988494da150>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True), raw_item=ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text='67 + 3 = 70\n', type='output_text', logprobs=None)], role='assistant', status='completed', type='message'), type='message_output_item')]


# **Asynchronous and Synchronous behaviour in run_sync and run_streamed:**

Aapka sawal hai ke `run_streamed` method asynchronous hai ya synchronous, jabke yeh diya gaya hai ke `run_sync` method synchronous hai. Synchronous ka matlab hai ke agar ek agent multiple tasks kar raha hai, to woh ek sequence mein kaam karega aur jab tak pehla task complete nahi hota, woh dosre task pe nahi jayega. Jabke asynchronous mein, agar pehle task ka result abhi nahi aaya, to agent sath sath doosre tasks bhi karta rahega bina pehle task ke result ka wait kiye. Ab hum code ko dekhte hain ke `run_streamed` kaunsa approach use karta hai.

### Code Analysis
1. **run_sync Method**:
   - Code mein `run_sync` method ko dekhein:
     ```python
     def run_sync(
         self,
         starting_agent: Agent[TContext],
         input: str | list[TResponseInputItem],
         **kwargs: Unpack[RunOptions[TContext]],
     ) -> RunResult:
         return asyncio.get_event_loop().run_until_complete(
             self.run(
                 starting_agent,
                 input,
                 context=kwargs.get("context"),
                 max_turns=kwargs.get("max_turns", DEFAULT_MAX_TURNS),
                 hooks=kwargs.get("hooks"),
                 run_config=kwargs.get("run_config"),
                 previous_response_id=kwargs.get("previous_response_id"),
             )
         )
     ```
   - Yeh method `self.run` (jo ke asynchronous hai) ko `asyncio.get_event_loop().run_until_complete` ke zariye call karta hai. Yeh ek synchronous wrapper hai jo async `run` method ko synchronous banata hai. Yani, yeh pura operation complete hone tak wait karta hai, jo synchronous behavior hai. Agar multiple tasks hain, to yeh ek ke baad ek execute honge.

2. **run_streamed Method**:
   - Ab `run_streamed` method ko dekhte hain:
     ```python
     def run_streamed(
         self,
         starting_agent: Agent[TContext],
         input: str | list[TResponseInputItem],
         **kwargs: Unpack[RunOptions[TContext]],
     ) -> RunResultStreaming:
         ...
         streamed_result._run_impl_task = asyncio.create_task(
             self._start_streaming(
                 starting_input=input,
                 streamed_result=streamed_result,
                 starting_agent=starting_agent,
                 max_turns=max_turns,
                 hooks=hooks,
                 context_wrapper=context_wrapper,
                 run_config=run_config,
                 previous_response_id=previous_response_id,
             )
         )
         return streamed_result
     ```
   - Yeh method ek `RunResultStreaming` object return karta hai, aur actual agent loop ko `_start_streaming` method ke zariye background mein chalata hai using `asyncio.create_task`. Yeh task asynchronous hai, kyunki `asyncio.create_task` ek coroutine ko background mein schedule karta hai aur control immediately wapas de deta hai bina task ke complete hone ka wait kiye.
   - `_start_streaming` method bhi asynchronous hai, kyunki isme `async def` use hua hai aur yeh `await` statements ke sath kaam karta hai, jaise:
     ```python
     async def _start_streaming(
         cls,
         starting_input: str | list[TResponseInputItem],
         streamed_result: RunResultStreaming,
         starting_agent: Agent[TContext],
         max_turns: int,
         hooks: RunHooks[TContext],
         context_wrapper: RunContextWrapper[TContext],
         run_config: RunConfig,
         previous_response_id: str | None,
     ):
         ...
         async for event in model.stream_response(...):
             streamed_result._event_queue.put_nowait(RawResponsesStreamEvent(data=event))
     ```
   - Yeh method events ko stream karta hai using an async generator (`async for`), jo ke asynchronous behavior hai. Yani, yeh events ko generate karta rahta hai jab tak woh available hote hain, aur sath hi sath doosre tasks (jaise input/output guardrails) ko bhi background mein handle karta hai.

3. **Asynchronous Behavior in run_streamed**:
   - `run_streamed` mein `_run_impl_task` ko `asyncio.create_task` ke sath background mein chalaya jata hai, jo ke asynchronous execution ko indicate karta hai.
   - Input guardrails bhi background mein chalte hain:
     ```python
     streamed_result._input_guardrails_task = asyncio.create_task(
         cls._run_input_guardrails_with_queue(...)
     )
     ```
   - Yeh dikhata hai ke guardrails aur streaming tasks sath sath chal sakte hain, jo asynchronous behavior hai.
   - `stream_response` method bhi async hai aur events ko stream karta hai, jo ke is baat ko reinforce karta hai ke `run_streamed` non-blocking hai aur multiple tasks concurrently handle kar sakta hai.

4. **Synchronous vs Asynchronous**:
   - **Synchronous (`run_sync`)**: Yeh method pura operation complete hone tak wait karta hai, yani ek task ke baad doosra task shuru hota hai. Yeh sequential execution hai.
   - **Asynchronous (`run_streamed`)**: Yeh method tasks ko background mein schedule karta hai (`asyncio.create_task`) aur events ko stream karta hai bina pura operation complete hone ke wait kiye. Yeh allow karta hai ke agent sath sath multiple tasks handle kare, jaise guardrails ko run karna aur events ko stream karna, bina pehle task ke result ka wait kiye.

### Nateeja
`run_streamed` method **asynchronous** hai. Yeh asynchronous scenario ko use karta hai, kyunki:
- Yeh tasks ko background mein schedule karta hai using `asyncio.create_task`.
- Yeh events ko asynchronously stream karta hai using `async for`.
- Yeh guardrails aur streaming tasks ko sath sath chalane deta hai, jo ke asynchronous behavior hai.

Jabke `run_sync` synchronous hai, kyunki yeh pura operation complete hone tak wait karta hai aur tasks ko sequence mein execute karta hai.

Agar aapko mazeed wazahat chahiye ya koi specific part ke bare mein poochna hai, to mujhe batayein!

In [6]:
agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant.",
    model=model1
)


result = Runner.run_streamed(
    agent,
    "Hi, My Name is Zain",
    run_config=config,
)

print(result.to_input_list())

[{'content': 'Hi, My Name is Zain', 'role': 'user'}]


----------
# ***) RunResult and RunResultStreaming properties and methods working:**

## **1. to_input_list():** input list LLM aur user ki conversation jo ik run mien hoti hai wo deta hai yani user ne sawal pocha aur llm ne jawab diya :

### Example:
```python
[{'content': 'Add 3 in 67 ', 'role': 'user'}, {'id': '__fake_id__', 'content': [{'annotations': [], 'text': '67 + 3 = 70\n', 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}]
```

------------
## **2. new_items ka result = MessageOutputItem:**

Aap ne mujhe ek code snippet diya hai jo ek `MessageOutputItem` ko represent karta hai. Yeh ek list hai jisme sirf ek item hai, aur yeh item `MessageOutputItem` class ka instance hai. Isme do main parts hain: ek `agent` aur ek `raw_item`. Mein aapko har ek cheez ko detail mein samjhaonga ke yeh kya hai aur kya kaam karti hai.

---

### **Pura Structure**
Yeh code ek list hai:

```python
[
    MessageOutputItem(
        agent=Agent(...),
        raw_item=ResponseOutputMessage(...),
        type='message_output_item'
    )
]
```

- **Kya hai?**: Yeh ek list hai jisme ek `MessageOutputItem` object hai.
- **Kaam**: Yeh agent ke output ko organize karne ke liye hai, jisme agent ki configuration aur uska generated message shamil hota hai.

Ab iske har part ko detail mein dekhte hain.

---

### **1. `MessageOutputItem`**
- **Kya hai?**: Yeh ek class ka instance hai jo agent ke dwara produce kiye gaye message ko hold karta hai.
- **Kaam**: Yeh ek structure provide karta hai jisme agent ki details aur uska output message ek sath store hota hai.

#### **Attributes:**
- **`agent`**: Ek `Agent` object jo message generate karne wala AI agent hai.
- **`raw_item`**: Ek `ResponseOutputMessage` object jo actual message ko contain karta hai.
- **`type='message_output_item'`**: Yeh specify karta hai ke yeh ek message output item hai.

---

### **2. `agent`**
- **Kya hai?**: Yeh `Agent` class ka instance hai jo ek AI agent ko represent karta hai. Is agent ko configure kiya gaya hai taake woh specific tasks perform kar sake, jaise arithmetic calculation ya user ke sawalon ka jawab dena.
- **Kaam**: Yeh agent user ke input ke base par responses generate karta hai. Is case mein, agent ka name "Assistant" hai aur uski instructions "You are a helpful assistant." hain, jo batata hai ke yeh user ki madad karne ke liye banaya gaya hai.

#### **Agent ke Attributes:**
- **`name='Assistant'`**  
  - **Kya hai?**: Agent ka name.  
  - **Kaam**: Yeh identify karta hai ke agent kaun hai. Yahan par "Assistant" ek simple identifier hai.

- **`instructions='You are a helpful assistant.'`**  
  - **Kya hai?**: Agent ko batane wali instructions.  
  - **Kaam**: Yeh agent ko uska role ya behavior define karta hai, yani yeh user ki madad karne wala assistant hai.

- **`handoff_description=None`**  
  - **Kya hai?**: Ek description jo shayad doosre agent ko control handoff karne ke liye hoti hai.  
  - **Kaam**: Yahan `None` hai, yani is feature ka use nahi ho raha.

- **`handoffs=[]`**  
  - **Kya hai?**: Doosre agents ki list jinko control handoff kiya ja sakta hai.  
  - **Kaam**: Khali list hai, yani koi handoff nahi hai.

- **`model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x00000202EEBB9A90>`**  
  - **Kya hai?**: Yeh ek OpenAI chat completion model ka instance hai jo agent responses generate karne ke liye use karta hai.  
  - **Kaam**: Yeh agent ka "brain" hai jo text ya answers produce karta hai.

- **`model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None)`**  
  - **Kya hai?**: Model ke liye settings ka ek object.  
  - **Kaam**: Yeh model ke behavior ko control karta hai (e.g., randomness, token limit), lekin yahan sab `None` hain, jo shayad default settings ko indicate karta hai.

- **`tools=[]`**  
  - **Kya hai?**: External tools ki list jo agent use kar sakta hai.  
  - **Kaam**: Khali hai, yani koi tools nahi hain; agent sirf model ke through kaam karta hai.

- **`mcp_servers=[]`**  
  - **Kya hai?**: Shayad model control panel servers ki list.  
  - **Kaam**: Khali hai, yani is feature ka use nahi ho raha.

- **`mcp_config={}`**  
  - **Kya hai?**: Model control panel ka configuration dictionary.  
  - **Kaam**: Khali hai, yani koi specific config nahi.

- **`input_guardrails=[]`**  
  - **Kya hai?**: Input ko validate ya filter karne wale rules ki list.  
  - **Kaam**: Khali hai, yani koi input checking nahi.

- **`output_guardrails=[]`**  
  - **Kya hai?**: Output ko validate ya filter karne wale rules ki list.  
  - **Kaam**: Khali hai, yani koi output checking nahi.

- **`output_type=None`**  
  - **Kya hai?**: Expected output ka type.  
  - **Kaam**: `None` hai, yani koi specific type define nahi kiya gaya.

- **`hooks=None`**  
  - **Kya hai?**: Custom behavior ya callbacks ke liye hooks.  
  - **Kaam**: `None` hai, yani koi hooks nahi.

- **`tool_use_behavior='run_llm_again'`**  
  - **Kya hai?**: Tool use hone par kya karna hai, yahan language model ko dobara run karna hai.  
  - **Kaam**: Chunki koi tools nahi hain, yeh relevant nahi hai.

- **`reset_tool_choice=True`**  
  - **Kya hai?**: Tool choice ko reset karne ka option.  
  - **Kaam**: Har use ke baad tool choice reset hota hai, lekin koi tools nahi hain to yeh bhi relevant nahi.

---

### **3. `raw_item`**
- **Kya hai?**: Yeh `ResponseOutputMessage` class ka instance hai jo agent ke dwara generate kiya gaya actual message hai.
- **Kaam**: Yeh message ki details (content, role, status) ko store karta hai.

#### **ResponseOutputMessage ke Attributes:**
- **`id='__fake_id__'`**  
  - **Kya hai?**: Message ka ID.  
  - **Kaam**: Yeh ek placeholder ID hai jo message ko identify karta hai.

- **`content=[ResponseOutputText(annotations=[], text='67 + 3 = 70\n', type='output_text')]`**  
  - **Kya hai?**: Message ka content, jo ek list mein ek `ResponseOutputText` object hai.  
  - **Kaam**: Yeh actual text "67 + 3 = 70" ko hold karta hai.  
    - **`annotations=[]`**: Koi additional notes ya metadata nahi.  
    - **`text='67 + 3 = 70\n'`**: Yeh calculation ka result hai jo agent ne diya.  
    - **`type='output_text'`**: Yeh batata hai ke content text type ka hai.

- **`role='assistant'`**  
  - **Kya hai?**: Message kis role se hai.  
  - **Kaam**: "assistant" indicate karta hai ke yeh agent ka response hai.

- **`status='completed'`**  
  - **Kya hai?**: Message ka status.  
  - **Kaam**: "completed" batata hai ke message fully generate ho chuka hai.

- **`type='message'`**  
  - **Kya hai?**: Yeh item ka type.  
  - **Kaam**: "message" specify karta hai ke yeh ek message hai.

---

### **Overall Kaam**
Yeh pura structure ek AI agent "Assistant" ke dwara generate kiye gaye message ko represent karta hai. Agent ek basic configuration ke sath set kiya gaya hai (koi tools, guardrails, ya hooks nahi). Isne ek simple arithmetic calculation "67 + 3 = 70" perform kiya aur uska result text message ke roop mein diya, jo ke sahi hai.

#### **Summary:**
- **Agent**: Ek "Assistant" jo OpenAI model use karta hai aur user ki madad ke liye banaya gaya hai.
- **Configuration**: Basic settings, koi extra features nahi.
- **Output**: Ek completed message jisme "67 + 3 = 70" likha hai.


---------
## **3. ModelSettings:**

## **ModelSettings Overview**
`ModelSettings` ek configuration hai jo batati hai ke AI model (jaise chatbot) kaise behave karega. Yeh ek class hai jismein 15 attributes hain, aur inki default value `None` hoti hai, yani agar aap kuch set nahi karte to model apni default setting use karta hai. Neeche har attribute ko detail mein samjha raha hoon.

---

### **1. `temperature`**
- **Kya Karta Hai**: Yeh model ke jawab ki "creativity" ko control karta hai. Zyada temperature se jawab creative aur random hote hain, kam temperature se jawab seedhe aur focused hote hain.
- **Code Example**:
  ```python
  model_settings = ModelSettings(temperature=0.8)
  ```
- **Use Case**:
  - **Situation**: Aap ek kahani likhne wala bot bana rahe hain.
  - **Setting**: `temperature=0.8` set karen taake bot naye aur dilchasp ideas de.
  - **Misal**: "Ek jungle mein ek anokha janwar tha..." (creative output).
- **Simple Bhasha Mein**: Temperature model ko batata hai ke kitna "mazedaar" ya "serious" hona hai.

---

### **2. `top_p`**
- **Meaning**: Yeh model ko batata hai ke kitne "probable" shabd ya jawab chunne hain. Iska value 0.0 se 1.0 tak hota hai.
- **Value Zyada Hone Ka Effect (e.g., `top_p=0.95`)**:
  - Model zyada options se jawab chunta hai, isliye jawab mein variety aati hai.
  - Example: "Mausam kaisa hai?" ka jawab ho sakta hai: "Mausam shandaar hai!" ya "Din sundar hai."
- **Value Kam Hone Ka Effect (e.g., `top_p=0.1`)**:
  - Model sirf sabse zyada likely jawab deta hai, isliye jawab predictable hote hain.
  - Example: "Mausam acha hai." (Har baar yahi jawab).
- **Code Example**:
  ```python
  model_settings = ModelSettings(top_p=0.95)  # Zyada variety
  # Ya
  model_settings = ModelSettings(top_p=0.1)   # Kam variety
  ```
- **Asaan Samajh**: Top_p model ko batata hai ke kitne jawabon ke "menu" se chunna hai. Zyada top_p = zyada choices, kam top_p = ek hi jawab.

---

### **3. `frequency_penalty`**
- **Meaning**: Yeh model ko same shabd ya sentences baar-baar bolne se rokta hai. Value usually 0.0 se 2.0 tak hoti hai.
- **Value Zyada Hone Ka Effect (e.g., `frequency_penalty=1.0`)**:
  - Model naye shabd use karta hai aur repeat bilkul nahi karta.
  - Example: "Mausam kaisa hai?" ka jawab: "Mausam acha hai." Doosra jawab: "Din shandaar hai." (repeat nahi).
- **Value Kam Hone Ka Effect (e.g., `frequency_penalty=0.0`)**:
  - Model shabd ya sentences repeat kar sakta hai.
  - Example: "Mausam acha hai." Doosra jawab: "Mausam acha hai." (same jawab).
- **Code Example**:
  ```python
  model_settings = ModelSettings(frequency_penalty=1.0)  # Kam repeat
  # Ya
  model_settings = ModelSettings(frequency_penalty=0.0)  # Zyada repeat
  ```
- **Asaan Samajh**: Frequency_penalty model ko kehta hai ke "nayi baat bol, purani mat dohra". Zyada penalty = naye shabd, kam penalty = repeat allowed.

---

### **4. `presence_penalty`**
- **Meaning**: Yeh model ko input ya pehle ke jawab ke shabd copy karne se rokta hai. Value usually 0.0 se 2.0 tak hoti hai.
- **Value Zyada Hone Ka Effect (e.g., `presence_penalty=1.0`)**:
  - Model naye shabd ya ideas deta hai, input ko copy nahi karta.
  - Example: Input: "Mausam acha hai." Jawab: "Din sundar lag raha hai." (input ke shabd nahi).
- **Value Kam Hone Ka Effect (e.g., `presence_penalty=0.0`)**:
  - Model input ke shabd ya ideas copy kar sakta hai.
  - Example: Input: "Mausam acha hai." Jawab: "Haan, mausam acha hai." (copy kiya).
- **Code Example**:
  ```python
  model_settings = ModelSettings(presence_penalty=1.0)  # Naye ideas
  # Ya
  model_settings = ModelSettings(presence_penalty=0.0)  # Copy allowed
  ```
- **Asaan Samajh**: Presence_penalty kehta hai "user ki baat ko copy mat kar, kuch naya bol". Zyada penalty = naya jawab, kam penalty = copy kar sakta hai.

---

### **5. `tool_choice`**
- **Meaning**: Yeh batata hai ke model ko kaunsa external tool (jaise calculator) use karna hai. Value ek string ya specific tool name hota hai.
- **Value Zyada (Set Kiya, e.g., `tool_choice="calculator")`**:
  - Model specific tool use karta hai aur usi se jawab deta hai.
  - Example: "2 + 2 kitna hai?" Jawab: "4" (calculator se).
- **Value Kam (Not Set, e.g., `tool_choice=None`)**:
  - Model apne text knowledge se jawab deta hai, tool nahi use karta.
  - Example: "2 + 2 kitna hai?" Jawab: "Char" (text-based).
- **Code Example**:
  ```python
  model_settings = ModelSettings(tool_choice="calculator")  # Tool use karen
  # Ya
  model_settings = ModelSettings(tool_choice=None)         # No tool
  ```
- **Asaan Samajh**: Tool_choice model ko batata hai ke koi bahari "machine" use karni hai ya nahi. Set kiya = tool use, nahi set = apna dimag use.

---

### **6. `parallel_tool_calls`**
- **Meaning**: Yeh model ko ek saath multiple tools use karne deta hai. Value `True` ya `False` hoti hai.
- **Value Zyada (e.g., `parallel_tool_calls=True`)**:
  - Model ek saath kai tools se data le sakta hai, jisse kaam jaldi hota hai.
  - Example: "Mausam aur traffic kaisa hai?" Jawab: "Mausam 25°C hai, traffic kam hai." (dono tools ek saath).
- **Value Kam (e.g., `parallel_tool_calls=False`)**:
  - Model ek-ek karke tools use karta hai, jisse time lagta hai.
  - Example: Pehle mausam, phir traffic ka jawab alag-alag.
- **Code Example**:
  ```python
  model_settings = ModelSettings(parallel_tool_calls=True)   # Ek saath tools
  # Ya
  model_settings = ModelSettings(parallel_tool_calls=False)  # Ek-ek tool
  ```
- **Asaan Samajh**: Parallel_tool_calls model ko kehta hai ke "ek saath do kaam kar" ya "ek ke baad doosra". True = jaldi kaam, False = dheema kaam.

---

### **7. `truncation`**
- **Meaning**: Yeh input ya jawab ko chhota karne ka rule set karta hai. Value ek number ya specific limit hoti hai.
- **Value Zyada (e.g., `truncation=50`)**:
  - Jawab ya input ko strictly 50 characters tak chhota karta hai.
  - Example: "Mausam kaisa hai?" Jawab: "Mausam acha hai, 25°C." (50 chars ke andar).
- **Value Kam (e.g., `truncation=10`)**:
  - Jawab bahut chhota ho jata hai, shayad incomplete.
  - Example: "Mausam acha..." (sirf 10 chars).
- **Code Example**:
  ```python
  model_settings = ModelSettings(truncation=50)  # 50 chars tak
  # Ya
  model_settings = ModelSettings(truncation=10)  # 10 chars tak
  ```
- **Asaan Samajh**: Truncation model ko kehta hai ke "itna hi bol, zyada nahi". Zyada value = thoda lamba jawab, kam value = chhota jawab.

---

### **8. `max_tokens`**
- **Meaning**: Yeh jawab mein maximum shabd ya "tokens" ki limit set karta hai. Value ek number hoti hai.
- **Value Zyada (e.g., `max_tokens=100`)**:
  - Model lamba jawab de sakta hai, lekin 100 shabd se zyada nahi.
  - Example: "Mausam kaisa hai?" Jawab: "Mausam bahut acha hai, 25°C hai, aur thodi hawa bhi chal rahi hai."
- **Value Kam (e.g., `max_tokens=10`)**:
  - Jawab chhota aur seedha hota hai.
  - Example: "Mausam acha hai."
- **Code Example**:
  ```python
  model_settings = ModelSettings(max_tokens=100)  # Lamba jawab
  # Ya
  model_settings = ModelSettings(max_tokens=10)   # Chhota jawab
  ```
- **Asaan Samajh**: Max_tokens model ko kehta hai ke "itne shabd tak bol". Zyada tokens = lamba, kam tokens = chhota.

---

### **9. `reasoning`**
- **Meaning**: Yeh model ko step-by-step sochne ya explanation dene ke liye kehta hai. Value `True` ya `False` hoti hai.
- **Value Zyada (e.g., `reasoning=True`)**:
  - Model jawab ke saath soch ka process batata hai.
  - Example: "2 + 2 kitna hai?" Jawab: "Pehle 2 lo, phir 2 jodo, to 4 banta hai."
- **Value Kam (e.g., `reasoning=False`)**:
  - Model sirf final jawab deta hai.
  - Example: "2 + 2 = 4."
- **Code Example**:
  ```python
  model_settings = ModelSettings(reasoning=True)   # Steps ke saath
  # Ya
  model_settings = ModelSettings(reasoning=False)  # Sirf jawab
  ```
- **Asaan Samajh**: Reasoning model ko kehta hai ke "batana kaise kiya". True = steps bataye, False = sirf result.

---

### **10. `metadata`**
- **Meaning**: Yeh extra information model ya system ko deta hai, jaise user ka naam ya session ID. Value ek dictionary hoti hai.
- **Value Zyada (e.g., `metadata={"user": "Rahul", "session": "123"}`)**:
  - Model ko context milta hai, aur jawab personalized ho sakta hai.
  - Example: "Rahul, mausam acha hai." (user ke naam ke saath).
- **Value Kam (e.g., `metadata=None`)**:
  - Model bina kisi extra info ke jawab deta hai.
  - Example: "Mausam acha hai." (generic jawab).
- **Code Example**:
  ```python
  model_settings = ModelSettings(metadata={"user": "Rahul"})  # Personal info
  # Ya
  model_settings = ModelSettings(metadata=None)              # No info
  ```
- **Asaan Samajh**: Metadata model ko kehta hai ke "yeh user kaun hai". Zyada info = personal jawab, kam info = simple jawab.

---

### **11. `store`**
- **Meaning**: Yeh batata hai ke baat-cheet ko save karna hai ya nahi. Value `True` ya `False` hoti hai.
- **Value Zyada (e.g., `store=True`)**:
  - Har sawal aur jawab save ho jata hai.
  - Example: "Mausam kaisa hai?" aur jawab database mein save ho jaye.
- **Value Kam (e.g., `store=False`)**:
  - Kuch bhi save nahi hota.
  - Example: Jawab deta hai aur bhool jata hai.
- **Code Example**:
  ```python
  model_settings = ModelSettings(store=True)   # Save karen
  # Ya
  model_settings = ModelSettings(store=False)  # Save nahi
  ```
- **Asaan Samajh**: Store model ko kehta hai ke "baat yaad rakho". True = save karta hai, False = bhool jata hai.

---

### **12. `include_usage`**
- **Meaning**: Yeh batata hai ke jawab ke saath resource usage (jaise kitne shabd use hue) dikhana hai ya nahi. Value `True` ya `False` hoti hai.
- **Value Zyada (e.g., `include_usage=True`)**:
  - Jawab ke saath usage stats milte hain.
  - Example: "Mausam acha hai. [10 tokens use hue]."
- **Value Kam (e.g., `include_usage=False`)**:
  - Sirf jawab milta hai, stats nahi.
  - Example: "Mausam acha hai."
- **Code Example**:
  ```python
  model_settings = ModelSettings(include_usage=True)   # Stats dikhaye
  # Ya
  model_settings = ModelSettings(include_usage=False)  # Stats nahi
  ```
- **Asaan Samajh**: Include_usage model ko kehta hai ke "kitna kaam kiya, woh bhi bata". True = stats deta hai, False = sirf jawab.

---

### **13. `extra_query`**
- **Meaning**: Yeh API call ke liye extra instructions ya parameters deta hai. Value ek dictionary hoti hai.
- **Value Zyada (e.g., `extra_query={"speed": "fast"}`)**:
  - API ko specific instructions milti hain, jaise jaldi jawab do.
  - Example: Jawab turant milta hai.
- **Value Kam (e.g., `extra_query=None`)**:
  - API normal tareeke se chalta hai.
  - Example: Jawab normal speed se aata hai.
- **Code Example**:
  ```python
  model_settings = ModelSettings(extra_query={"speed": "fast"})  # Fast mode
  # Ya
  model_settings = ModelSettings(extra_query=None)              # Normal mode
  ```
- **Asaan Samajh**: Extra_query model ko kehta hai ke "API ko yeh extra baat bol". Zyada query = special mode, kam query = normal mode.

---

### **14. `extra_body`**
- **Meaning**: Yeh API request ke body mein extra data jodta hai. Value ek dictionary hoti hai.
- **Value Zyada (e.g., `extra_body={"priority": "high"}`)**:
  - API ko extra info milti hai, jaise urgent request.
  - Example: Jawab pehle process hota hai.
- **Value Kam (e.g., `extra_body=None`)**:
  - API normal request ke saath chalta hai.
  - Example: Jawab normal queue mein aata hai.
- **Code Example**:
  ```python
  model_settings = ModelSettings(extra_body={"priority": "high"})  # Urgent
  # Ya
  model_settings = ModelSettings(extra_body=None)                 # Normal
  ```
- **Asaan Samajh**: Extra_body model ko kehta hai ke "request ke saath yeh jod do". Zyada body = special request, kam body = simple request.

---

### **15. `extra_headers`**
- **Meaning**: Yeh API call ke liye extra HTTP headers (jaise security keys) deta hai. Value ek dictionary hoti hai.
- **Value Zyada (e.g., `extra_headers={"api_key": "secret"}`)**:
  - API secure aur verified hoti hai.
  - Example: Sirf sahi key wale jawab dete hain.
- **Value Kam (e.g., `extra_headers=None`)**:
  - API bina extra security ke chalta hai.
  - Example: Normal API call.
- **Code Example**:
  ```python
  model_settings = ModelSettings(extra_headers={"api_key": "secret"})  # Secure
  # Ya
  model_settings = ModelSettings(extra_headers=None)                  # Normal
  ```
- **Asaan Samajh**: Extra_headers model ko kehta hai ke "API ko yeh security pass do". Zyada headers = secure call, kam headers = simple call.

---



## **4. raw_response: raw_responses mein agent loop ke dauraan jitni baar LLM ne responses diye, woh saare responses store hote hain, chahe woh ek agent ke ho ya multiple agents ke.**

---

### **Raw Response Overview**
Yeh `raw_response` ek structured output hai jo model ke jawab aur uske metadata (jaise token usage) ko represent karta hai. Iska structure ek list ke andar ek `ModelResponse` object hai, jismein multiple fields hain. Har field ka kaam alag-alag hai. Chalo, ek-ek karke samajhte hain:

```python
[ModelResponse(
    output=[ResponseOutputMessage(
        id='__fake_id__',
        content=[ResponseOutputText(
            annotations=[],
            text='Mausam ki jaankaari ke liye, mujhe aapko yeh bataana hoga ki aap kahaan hain. Kripya mujhe apne shehar ya ilaake ka naam bataayein, taaki main aapko sahi jaankaari de sakun.\n',
            type='output_text'
        )],
        role='assistant',
        status='completed',
        type='message'
    )],
    usage=Usage(
        requests=1,
        input_tokens=19,
        input_tokens_details=InputTokensDetails(cached_tokens=0),
        output_tokens=61,
        output_tokens_details=OutputTokensDetails(reasoning_tokens=0),
        total_tokens=80
    ),
    response_id=None
)]
```

---

### **1. `ModelResponse`**
- **Kya Hai?**: Yeh pura response object hai jo model ke jawab ko represent karta hai. Yeh batata hai ke model ne kya output diya, kitne tokens use hue, aur response ka status kya hai.
- **Asaan Alfaaz Mein**: Yeh ek "dabba" hai jismein model ka pura jawab aur uski details (jaise kitna kaam kiya) rakhi hain.

---

### **2. `output` Field**
- **Kya Hai?**: Yeh ek list hai jismein model ke actual jawab hote hain. Is case mein, ek hi jawab hai jo `ResponseOutputMessage` object ke roop mein hai.
- **Details**:
  - `output=[ResponseOutputMessage(...)]`: Yeh batata hai ke model ne ek message diya hai. Agar model multiple messages deta (jaise chat history), to yahan aur items hote.
- **Asaan Alfaaz Mein**: Yeh model ka asli jawab hai, jo ek ya zyada messages ka collection ho sakta hai. Yahan sirf ek message hai.

---

### **3. `ResponseOutputMessage`**
- **Kya Hai?**: Yeh ek single message object hai jo model ne generate kiya. Ismein message ki details hoti hain, jaise content, role, aur status.
- **Sub-Fields**:
  1. **`id='__fake_id__'`**:
     - **Kya Hai?**: Yeh message ka unique identifier hai. Yahan `__fake_id__` diya gaya, jo shayad testing ya placeholder ke liye hai.
     - **Asaan Alfaaz Mein**: Yeh message ka "naam" ya number hai jo usse alag karta hai. Yahan fake ID diya gaya, yani real nahi.
  2. **`content=[ResponseOutputText(...)]`**:
     - **Kya Hai?**: Yeh message ka actual text ya data hai. Yahan ek `ResponseOutputText` object hai jismein model ka jawab hai.
     - **Asaan Alfaaz Mein**: Yeh woh text hai jo model ne bola, jaise "Mausam ki jaankaari ke liye...".
  3. **`role='assistant'`**:
     - **Kya Hai?**: Yeh batata hai ke message kisne diya. `assistant` ka matlab hai ke yeh AI model ka jawab hai (user ke bajaye).
     - **Asaan Alfaaz Mein**: Yeh batata hai ke yeh bot ka jawab hai, na ki aapka sawal.
  4. **`status='completed'`**:
     - **Kya Hai?**: Yeh batata hai ke message ka processing pura ho gaya hai.
     - **Asaan Alfaaz Mein**: Iska matlab hai ke model ne apna kaam khatam kar diya, jawab ready hai.
  5. **`type='message'`**:
     - **Kya Hai?**: Yeh batata hai ke yeh ek message-type output hai (aur doosre types jaise tool calls bhi ho sakte hain).
     - **Asaan Alfaaz Mein**: Yeh kehta hai ke yeh ek normal text jawab hai.

---

### **4. `ResponseOutputText`**
- **Kya Hai?**: Yeh message ka actual text content hai, jo model ne generate kiya.
- **Sub-Fields**:
  1. **`annotations=[]`**:
     - **Kya Hai?**: Yeh empty list hai jo text ke saath koi extra information (jaise links, formatting) store kar sakti hai. Yahan koi annotations nahi hain.
     - **Asaan Alfaaz Mein**: Yeh text ke saath koi special tags ya notes hote, lekin yahan kuch nahi hai.
  2. **`text='Mausam ki jaankaari ke liye...'`**:
     - **Kya Hai?**: Yeh model ka actual jawab hai: "Mausam ki jaankaari ke liye, mujhe aapko yeh bataana hoga ki aap kahaan hain. Kripya mujhe apne shehar ya ilaake ka naam bataayein, taaki main aapko sahi jaankaari de sakun."
     - **Asaan Alfaaz Mein**: Yeh woh baat hai jo bot ne boli. Model kehta hai ke mausam batane ke liye location chahiye.
  3. **`type='output_text'`**:
     - **Kya Hai?**: Yeh batata hai ke content ek text message hai (aur doosre types jaise images ya code bhi ho sakte hain).
     - **Asaan Alfaaz Mein**: Yeh kehta hai ke yeh jawab text ke roop mein hai.

---

### **5. `usage` Field**
- **Kya Hai?**: Yeh batata hai ke model ne kitne resources (tokens) use kiye request process karne mein.
- **Sub-Fields**:
  1. **`requests=1`**:
     - **Kya Hai?**: Yeh batata hai ke ek request bheja gaya tha.
     - **Asaan Alfaaz Mein**: Aapne model ko ek baar sawal poocha.
  2. **`input_tokens=19`**:
     - **Kya Hai?**: Yeh aapke prompt ("Mausam ki jaankaari") ke tokens (words ya subwords) ki ginti hai.
     - **Asaan Alfaaz Mein**: Aapke sawal mein 19 "shabd" (tokens) the.
  3. **`input_tokens_details=InputTokensDetails(cached_tokens=0)`**:
     - **Kya Hai?**: Yeh input tokens ke details deta hai. `cached_tokens=0` ka matlab hai ke koi bhi input tokens cached (pehle se save) nahi the.
     - **Asaan Alfaaz Mein**: Model ne aapka sawal fresh process kiya, pehle se koi data use nahi kiya.
  4. **`output_tokens=61`**:
     - **Kya Hai?**: Yeh model ke jawab ke tokens ki ginti hai.
     - **Asaan Alfaaz Mein**: Model ke jawab mein 61 "shabd" (tokens) hain.
  5. **`output_tokens_details=OutputTokensDetails(reasoning_tokens=0)`**:
     - **Kya Hai?**: Yeh output tokens ke details deta hai. `reasoning_tokens=0` ka matlab hai ke model ne koi reasoning steps (jaise step-by-step soch) use nahi kiye.
     - **Asaan Alfaaz Mein**: Model ne sidha jawab diya, bina kisi extra soch ke.
  6. **`total_tokens=80`**:
     - **Kya Hai?**: Yeh input aur output tokens ka total (19 + 61 = 80).
     - **Asaan Alfaaz Mein**: Total kaam mein 80 "shabd" lage.

---

### **6. `response_id=None`**
- **Kya Hai?**: Yeh response ka unique ID hota hai, lekin yahan `None` hai, jo shayad isliye kyunki yeh test ya fake response hai.
- **Asaan Alfaaz Mein**: Yeh response ka "naam" hota, lekin yahan koi naam nahi diya gaya.

---

### **Overall Kya Hua?**
- **Prompt**: Aapne model se poocha, "Mausam ki jaankaari."
- **Jawab**: Model ne kaha, "Mujhe aapka shehar ya location batao taaki main sahi mausam ki jaankaari de sakun."
- **Stats**: Aapke sawal mein 19 tokens lage, jawab mein 61 tokens, aur total 80 tokens use hue.
- **Status**: Model ne kaam pura kiya (`completed`), aur jawab ek assistant ke roop mein text format mein aaya.
- **Annotations**: Koi extra tags ya info nahi di gayi.
- **Reasoning**: Model ne koi step-by-step soch ka use nahi kiya (`reasoning_tokens=0`).

---

### **Asaan Summary**
Yeh response ek "dabba" hai jismein:
1. **Model ka Jawab**: "Mausam batane ke liye location do."
2. **Details**: Yeh jawab assistant ne diya, kaam pura ho gaya, aur text format mein hai.
3. **Stats**: Sawal mein 19 shabd, jawab mein 61 shabd, total 80 shabd lage.
4. **Extra Info**: Koi special tags ya reasoning nahi tha, aur response ka koi unique ID nahi diya gaya.
-----------


# **5. Last_agent:**


### **Agent Object Overview**
Yeh `Agent` object ek configuration hai jo batata hai ke aapka AI agent (jo ek weather assistant hai) kaise kaam karega. Iska structure complex hai, lekin main ise asaan karke samjhaunga:

```python
Agent(
    name='Assistant',
    instructions='You are Powerfull weather Assistant use tool for answering any ewather releated query',
    handoff_description=None,
    handoffs=[],
    model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x00000218FF7C33D0>,
    model_settings=ModelSettings(...),
    tools=[FunctionTool(...)],
    mcp_servers=[],
    mcp_config={},
    input_guardrails=[],
    output_guardrails=[],
    output_type=None,
    hooks=None,
    tool_use_behavior='run_llm_again',
    reset_tool_choice=True
)
```

---

### **1. `name='Assistant'`**
- **Kya Hai?**: Yeh agent ka naam hai jo aapne diya hai.
- **Asaan Alfaaz Mein**: Yeh batata hai ke is AI ka naam "Assistant" hai. Ise aap kuch bhi naam de sakte hain, jaise "WeatherBot".

---

### **2. `instructions='You are Powerfull weather Assistant use tool for answering any ewather releated query'`**
- **Kya Hai?**: Yeh agent ko batata hai ke uska kaam kya hai. Yahan instructions kehti hain ke yeh ek "powerful weather assistant" hai jo weather-related sawalon ke jawab dene ke liye tools ka use karega.
- **Asaan Alfaaz Mein**: Yeh agent ko bolta hai, "Tum ek weather expert ho, aur jab koi mausam ka sawal puche, to tool use karke jawab do." (Note: "ewather" shayad typo hai, "weather" hona chahiye.)

---

### **3. `handoff_description=None`**
- **Kya Hai?**: Yeh field batata hai ke agar agent kaam na kar sake, to kya doosre agent ko kaam dena hai. Yahan `None` hai, yani koi handoff description nahi diya.
- **Asaan Alfaaz Mein**: Yeh kehta hai ke agar yeh agent jawab na de sake, to koi backup plan nahi hai.

---

### **4. `handoffs=[]`**
- **Kya Hai?**: Yeh ek list hai jo doosre agents ke saath handoff rules define karti hai. Yahan empty list (`[]`) hai, yani koi handoff nahi hai.
- **Asaan Alfaaz Mein**: Yeh bolta hai ke is agent ke liye koi doosra agent backup ke roop mein nahi hai.

---

### **5. `model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x...>`**
- **Kya Hai?**: Yeh batata hai ke agent kis AI model ka use kar raha hai. Yahan `OpenAIChatCompletionsModel` hai, jo shayad Gemini API (`gemini-2.0-flash`) ya OpenAI model se juda hai.
- **Asaan Alfaaz Mein**: Yeh agent ka "dimag" hai, jo sawalon ke jawab deta hai. Is case mein, yeh ek specific model hai jo aapne configure kiya.

---

### **6. `model_settings=ModelSettings(...)`**
- **Kya Hai?**: Yeh `ModelSettings` object hai jo model ke behavior ko control karta hai, jaise temperature, reasoning, metadata, etc. Yahan sab fields `None` hain, yani default settings use ho rahi hain.
- **Sub-Fields**:
  - `temperature=None`, `top_p=None`, `frequency_penalty=None`, `presence_penalty=None`, etc.: Yeh sab model ke creative ya repetitive behavior ke settings hain. `None` ka matlab hai ke model apne default values use karega.
  - `reasoning=None`: Koi step-by-step reasoning nahi hai.
  - `metadata=None`: Koi extra info nahi di gayi.
  - `store=None`, `include_usage=None`: Response save ya usage stats include nahi kiye.
  - `extra_query=None`, `extra_body=None`, `extra_headers=None`: Koi additional API settings nahi hain.
- **Asaan Alfaaz Mein**: Yeh settings batati hain ke model kaise jawab dega. Yahan sab default pe chhod diya gaya, yani model apne standard tareeke se kaam karega.

---

### **7. `tools=[FunctionTool(...)]`**
- **Kya Hai?**: Yeh ek list hai jo agent ke paas available tools ko define karti hai. Yahan ek `FunctionTool` hai jo `fetch_weather` ke naam se hai.
- **Sub-Fields of `FunctionTool`**:
  1. **`name='fetch_weather'`**:
     - **Kya Hai?**: Tool ka naam hai, jo weather data fetch karne ke liye hai.
     - **Asaan Alfaaz Mein**: Yeh ek "machine" hai jo mausam ki jaankaari la sakti hai.
  2. **`description=''`**:
     - **Kya Hai?**: Tool ka description empty hai, jo batata hai ke tool kya karta hai. Empty hone se model ko thodi confusion ho sakti hai.
     - **Asaan Alfaaz Mein**: Yeh tool ke baare mein koi info nahi deta, jo shayad issue ho sakta hai.
  3. **`params_json_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], ...}`**:
     - **Kya Hai?**: Yeh JSON schema batata hai ke `fetch_weather` tool ko ek `city` parameter chahiye, jo ek string hona chahiye.
     - **Asaan Alfaaz Mein**: Tool ko kaam karne ke liye city ka naam chahiye, jaise "Delhi" ya "Mumbai".
  4. **`on_invoke_tool=<function ...>`**:
     - **Kya Hai?**: Yeh function hai jo tab chalta hai jab tool use hota hai.
     - **Asaan Alfaaz Mein**: Yeh batata hai ke tool kaise kaam karega jab model usse call karega.
  5. **`strict_json_schema=True`**:
     - **Kya Hai?**: Yeh ensure karta hai ke tool ka input strictly JSON schema ke mutabik ho.
     - **Asaan Alfaaz Mein**: Yeh kehta hai ke tool ko sirf wahi data do jo schema mein define hai (city ka naam).

- **Asaan Alfaaz Mein**: Agent ke paas ek tool hai jo mausam ki jaankaari la sakta hai, lekin usse city ka naam chahiye. Tool ka description khali hai, jo model ke liye thodi dikkat paida kar sakta hai.

---

### **8. `mcp_servers=[]`**
- **Kya Hai?**: Yeh list model control plane (MCP) servers ko define karti hai, jo agent ke behavior ko manage karte hain. Yahan empty hai.
- **Asaan Alfaaz Mein**: Yeh kehta hai ke koi external server agent ko control nahi kar raha.

---

### **9. `mcp_config={}`**
- **Kya Hai?**: Yeh model control plane ke settings hain. Empty dictionary ka matlab hai ke koi special config nahi hai.
- **Asaan Alfaaz Mein**: Agent ke liye koi extra settings nahi hain.

---

### **10. `input_guardrails=[]`**
- **Kya Hai?**: Yeh list input ke liye guardrails (rules) define karti hai, jo check karte hain ke user ka input valid hai ya nahi. Empty hone ka matlab hai ke koi guardrails nahi hain.
- **Asaan Alfaaz Mein**: Agent user ke sawal ko check nahi karta, seedha accept kar lega.

---

### **11. `output_guardrails=[]`**
- **Kya Hai?**: Yeh list output ke liye guardrails define karti hai, jo model ke jawab ko validate karte hain. Empty hone ka matlab hai ke koi output check nahi hai.
- **Asaan Alfaaz Mein**: Agent jo jawab dega, usse koi rule check nahi karega.

---

### **12. `output_type=None`**
- **Kya Hai?**: Yeh batata hai ke model ka output kaisa hona chahiye (jaise structured dictionary ya plain text). `None` ka matlab hai ke koi specific output type set nahi kiya.
- **Asaan Alfaaz Mein**: Agent ka jawab kisi fixed format mein nahi hoga, jo model decide karega.

---

### **13. `hooks=None`**
- **Kya Hai?**: Yeh custom functions ya hooks hote hain jo agent ke kaam ke darmiyan chal sakte hain. `None` ka matlab hai ke koi hooks nahi hain.
- **Asaan Alfaaz Mein**: Agent ke kaam mein koi extra steps nahi jode gaye.

---

### **14. `tool_use_behavior='run_llm_again'`**
- **Kya Hai?**: Yeh batata hai ke tool use karne ke baad kya karna hai. `'run_llm_again'` ka matlab hai ke tool ka result milne ke baad model dobara chalega taake final jawab de sake.
- **Asaan Alfaaz Mein**: Agar tool (fetch_weather) use hota hai, to model uske result ke saath dobara sochta hai aur jawab deta hai.

---

### **15. `reset_tool_choice=True`**
- **Kya Hai?**: Yeh batata hai ke har request ke baad tool choice reset karna hai ya nahi. `True` ka matlab hai ke har baar model naye tareeke se tool chunega.
- **Asaan Alfaaz Mein**: Har sawal ke liye model naye sira se decide karega ke tool use karna hai ya nahi.

---

### **Overall Kya Hai?**
Yeh `Agent` object ek weather assistant ka configuration hai, jo:
- **Naam**: "Assistant" hai.
- **Kaam**: Mausam ke sawalon ke jawab deta hai aur `fetch_weather` tool ka use karta hai.
- **Tool**: Ek tool hai jo city ke naam ke saath mausam ki jaankaari la sakta hai.
- **Settings**: Sab default pe hain (temperature, reasoning, metadata, etc., sab `None`).
- **Behavior**: Tool use karne ke baad model dobara chalega aur har request ke liye tool choice reset hoga.
- **Guardrails**: Koi input ya output checks nahi hain.
- **Output**: Koi fixed format nahi hai, model jo dega woh accept hoga.

---

### **Asaan Summary**
Yeh ek weather assistant ka "blueprint" hai:
- **Naam aur Kaam**: Yeh "Assistant" hai jo mausam ke sawalon ke liye tool use karta hai.
- **Tool**: Ek tool hai jo city ke naam se mausam batata hai.
- **Settings**: Sab default pe hain, koi special configuration nahi.
- **Rules**: Koi input/output checks nahi, aur tool use ke baad model dobara chalega.
- **Output**: Jawab ka format model pe depend karta hai.

--------


## **6. Turn(action) Confusion:**

Aapka sawal OpenAI-Agents SDK ke context mein `max_turns` parameter aur agent ke loop ke behavior se related hai. Specifically, aap janana chahte hain ke jab agent ek loop mein baar-baar LLM (Language Model) ko call karta hai, to kya yeh LLM requests bhi `max_turns` mein count hote hain ya nahi. Main isko asaan alfaaz mein aur SDK ke behavior ke hisaab se explain karunga.

---

### **Short Answer**
OpenAI-Agents SDK mein `max_turns` parameter **agent ke high-level actions** (ya "turns") ko count karta hai, na ki har LLM request ko. Yani, agar agent ek turn ke andar multiple LLM calls karta hai (jaise tool use ya reasoning ke liye), to yeh sab ek hi turn mein count hote hain, jab tak agent ka action complete na ho ya naya turn start na ho.

---

### **Detailed Explanation**

#### **1. `max_turns` Kya Hai?**
- OpenAI-Agents SDK mein `max_turns` ek parameter hai jo `RunConfig` mein set kiya jata hai. Yeh batata hai ke agent kitne **turns** (ya steps) tak loop chala sakta hai apne task ko complete karne ke liye.
- Ek "turn" typically ek complete cycle hota hai jismein agent:
  1. User input ya context leta hai.
  2. LLM ko call karta hai (ek ya zyada baar).
  3. Tools ka use karta hai (agar zaroori ho).
  4. Ek interim ya final output deta hai.
- `max_turns` ka limit isliye hota hai taake agent infinite loop mein na fase.

#### **2. Turn vs. LLM Request**
- **Turn**: Ek turn ek logical step hai jismein agent apna kaam aage badhata hai. Yeh include kar sakta hai:
  - Ek ya zyada LLM calls.
  - Tool invocations (jaise `fetch_weather`).
  - Internal reasoning ya decision-making.
- **LLM Request**: Yeh ek single API call hai jo model (jaise Gemini ya OpenAI) ke server ko jata hai. Ek turn ke andar multiple LLM requests ho sakte hain, jaise:
  - Pehle LLM se prompt ka jawab lena.
  - Tool use ka decision lene ke liye dobara LLM call.
  - Tool ke result ke baad final output ke liye ek aur LLM call.

- **Key Point**: SDK mein `max_turns` **turns** ko count karta hai, na ki har LLM request ko. Yani, agar ek turn mein agent 3 LLM calls karta hai (jaise tool call, reasoning, aur final output ke liye), to yeh sab ek hi turn mein count hoga.

#### **3. Agent Loop Ka Behavior**
OpenAI-Agents SDK mein agent ka loop aise kaam karta hai:
1. **Input Processing**: Agent user input ya previous context ko process karta hai.
2. **LLM Call**: Agent LLM ko call karta hai jawab ya action ke liye.
3. **Tool Use (Optional)**: Agar LLM tool use ka suggestion deta hai (jaise `fetch_weather`), to agent tool ko invoke karta hai aur shayad dobara LLM call karta hai tool ke result ko process karne ke liye.
4. **Output**: Jab turn complete hota hai (ya to final jawab mil jata hai ya interim state update hota hai), to agent agle turn pe jata hai ya ruk jata hai.

- **Example**:
  - Aapka prompt: "Delhi ka mausam kya hai?"
  - **Turn 1**:
    - Agent LLM ko call karta hai aur model kehta hai, "Mujhe city ke liye tool use karna hoga."
    - Agent `fetch_weather` tool ko call karta hai (ek aur LLM call ho sakta hai tool ke input ke liye).
    - Tool se result aata hai, aur agent ek final LLM call karta hai jawab format karne ke liye: "Delhi mein aaj 30°C hai."
    - Yeh sab **ek turn** mein hota hai.
  - Agar agent ko aur kaam karna ho (jaise user bolta hai, "Aur details do"), to yeh **Turn 2** hoga.
  - `max_turns=5` ka matlab hai ke agent aise 5 cycles (turns) tak chal sakta hai.

- **LLM Calls in Turn**: Ek turn mein multiple LLM calls ho sakte hain (jaise tool use ke liye, reasoning ke liye, ya output formatting ke liye), lekin yeh sab ek hi turn ke hisaab se count hote hain.

#### **4. SDK Ke Hisaab Se**
OpenAI-Agents SDK ke GitHub repo (https://github.com/openai/openai-agents-python) aur documentation ke mutabik:
- `max_turns` ka control `Runner.run` method mein hota hai, jo agent ke loop ko manage karta hai.
- Har turn mein `RunContext` update hota hai, aur turn tab complete hota hai jab agent ek logical step pura karta hai (jaise tool result ke baad jawab dena).
- LLM requests ka count `usage` field mein track hota hai (jaise `input_tokens`, `output_tokens`), lekin yeh `max_turns` se alag hai.
- Agar aapka agent `tool_use_behavior='run_llm_again'` set karta hai (jo aapke pichle code mein tha), to tool use ke baad LLM dobara call hota hai, lekin yeh bhi usi turn ka hissa hota hai.

#### **5. Aapke Sawal Ka Jawab**
> Agar agent baar-baar LLM call kar raha hai, to kya LLM requests bhi turn mein count hoti hain?
- **Nahi**, LLM requests alag se `max_turns` mein count nahi hoti. Ek turn ke andar kitne bhi LLM calls ho, woh sab usi turn ke andar aate hain. `max_turns` sirf high-level steps (turns) ko limit karta hai, jo agent ke logical actions (jaise input process, tool use, output generate) ke hisaab se count hote hain.
- **Example**:
  - Agar agent ek turn mein 3 LLM calls karta hai (prompt ke liye, tool call ke liye, aur final jawab ke liye), to yeh **ek turn** count hoga.
  - Agar agent 5 turns chalta hai aur har turn mein 2 LLM calls hote hain, to total 10 LLM calls honge, lekin `max_turns=5` ke hisaab se sirf 5 turns count honge.

---

### **How to Verify?**
Agar aap confirm karna chahte hain ke `max_turns` kaise count hota hai:
1. **Set `max_turns`**:
   ```python
   config = RunConfig(
       model=model,
       model_provider=external_client,
       tracing_disabled=True,
       max_turns=2  # Limit to 2 turns
   )
   ```
2. **Logging Enable Karen**:
   ```python
   import logging
   logging.basicConfig(level=logging.DEBUG)
   ```
   - Yeh SDK ke internal logs dikhayega, jismein har turn aur LLM call ka detail hoga.
3. **Usage Check Karen**:
   - Response mein `usage` field se total LLM requests aur tokens check karen:
     ```python
     print(result.usage)
     ```
   - Yeh batayega kitne LLM calls hue, lekin `max_turns` se alag rahega.

---

### **Asaan Summary**
- **Turn**: Ek turn ek pura step hai jismein agent input leta hai, LLM call(s) karta hai, tools use karta hai, aur output deta hai.
- **LLM Request**: Ek single API call hai jo model ko jata hai. Ek turn mein multiple LLM calls ho sakte hain.
- **Max Turns**: `max_turns` sirf turns ko count karta hai, na ki LLM requests ko. Yani, ek turn ke andar kitne bhi LLM calls ho, woh ek hi turn mein aate hain.
- **Aapke Case Mein**: Agar aapka agent loop mein baar-baar LLM call kar raha hai, to yeh calls usi turn ka hissa hain aur `max_turns` mein alag se count nahi hote.

-----------


# **Max_Turn understanding in this code
```python
from dotenv import load_dotenv
import os
import asyncio
from pydantic import BaseModel
from agents import (
    Agent,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig,
    Runner,
    ModelSettings,
    function_tool,
)

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
base_url = os.getenv("BASE_URL")

external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/",  # Corrected base_url
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client,
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True,
)

@function_tool
def fetch_weather(city: str) -> str:
    return f"Weather of {city} is Cloudy and Temperature is 25C"

weather_agent = Agent(
    name="Weather_Agent",
    instructions="You are a Weather Assistant. Use the fetch_weather tool to answer weather-related queries.",
    model=model,
    tools=[fetch_weather],
)

agent = Agent(
    name="Assistant",
    instructions="You are a Powerful Assistant. Handoff weather-related queries to Weather_Agent.",
    model=model,
    handoffs=[weather_agent]
)

async def main():
    result = Runner.run_streamed(
        agent,
        "What's the weather in Gujar Khan",
        run_config=config,
        max_turns=2
    )
    async for event in result.stream_events():
        print(event)


if __name__ == "__main__":
    asyncio.run(main())
```
:
### **Error Ka Breakdown**
- **Error**: `MaxTurnsExceeded` error tab aata hai jab agent `max_turns` se zyada turns (logical steps) leta hai. Aapne `max_turns=2` set kiya hai, lekin agent 2 se zyada turns le raha hai.
- **Aapka Sawal**: Aapko lagta hai ke agent sirf 2 turns le raha hai (main agent ka handoff aur weather agent ka tool use), lekin error kyun aa raha hai?
- **Problem**: Agent ka workflow aapke soch se zyada complex hai, aur har action (handoff, tool use, final output) alag-alag turns mein count ho raha hai.

---

### **Agent Kitne Turns Le Raha Hai?**
Chalo, aapke code ke flow ko analyze karte hain aur dekhte hain ke agent kitne turns le raha hai jab query hai: "What's the weather in Gujar Khan".

#### **Code Flow**
1. **Main Agent**:
   - Name: `Assistant`
   - Instructions: "Handoff weather-related queries to Weather_Agent."
   - Handoff: `weather_agent`
   - Kaam: Query ko process karta hai aur weather-related hone pe `Weather_Agent` ko handoff karta hai.

2. **Weather Agent**:
   - Name: `Weather_Agent`
   - Instructions: "Use the fetch_weather tool to answer weather-related queries."
   - Tool: `fetch_weather`
   - Kaam: `fetch_weather` tool ko call karta hai aur result return karta hai.

3. **Run Configuration**:
   - `max_turns=2`: Agent ke total turns ka limit 2 hai.
   - `Runner.run_streamed`: Streaming mode mein events generate karta hai.

4. **Query**: "What's the weather in Gujar Khan"
   - Main agent is query ko process karega, handoff karega, aur weather agent tool use karke jawab dega.

#### **Turn-by-Turn Breakdown**
OpenAI-Agents SDK mein har **turn** ek logical step hai jismein agent input process karta hai, LLM call karta hai, tools use karta hai, ya handoff karta hai. Chalo, turn-by-turn dekhte hain:

1. **Turn 1: Main Agent (Handoff Decision)**:
   - Main agent query ("What's the weather in Gujar Khan") ko process karta hai.
   - Instructions ke mutabik, yeh weather-related query hai, isliye agent `Weather_Agent` ko handoff karta hai.
   - Is process mein:
     - LLM call hota hai query ko analyze karne ke liye.
     - Handoff decision liya jata hai.
   - **Output**: Shayad ek intermediate output jaise "Handoff to Weather_Agent".
   - **Turn Count**: Yeh **1st turn** hai.

2. **Turn 2: Weather Agent (Tool Use Decision)**:
   - `Weather_Agent` handoff se query context leta hai.
   - Instructions ke mutabik, yeh `fetch_weather` tool use karta hai.
   - Is process mein:
     - LLM call hota hai tool use ka decision lene ke liye (e.g., "fetch_weather(city='Gujar Khan')").
   - **Output**: Tool call ka decision (lekin abhi tool result nahi aaya).
   - **Turn Count**: Yeh **2nd turn** hai.

3. **Turn 3: Weather Agent (Tool Result Processing)**:
   - `fetch_weather` tool call hota hai aur result return karta hai: "Weather of Gujar Khan is Cloudy and Temperature is 25C".
   - Weather agent is tool result ko process karta hai aur final output generate karta hai.
   - Is process mein:
     - LLM call hota hai tool result ko format karne ke liye (e.g., final jawab banane ke liye).
   - **Output**: Final output jaise "Weather of Gujar Khan is Cloudy and Temperature is 25C".
   - **Turn Count**: Yeh **3rd turn** hai.

#### **Kyun Error Aaya?**
- Aapne `max_turns=2` set kiya, lekin agent **3 turns** le raha hai:
  - Turn 1: Main agent ka handoff.
  - Turn 2: Weather agent ka tool use decision.
  - Turn 3: Weather agent ka tool result processing.
- Jab agent 3rd turn shuru karta hai, SDK check karta hai ke `max_turns=2` exceed ho gaya, aur `MaxTurnsExceeded` error throw karta hai.

---

### **Kyun Aapko Laga Ke 2 Turns Hain?**
Aapko shayad laga ke:
- Turn 1: Main agent ka handoff.
- Turn 2: Weather agent ka tool use aur final output.
Lekin OpenAI-Agents SDK mein **tool use ka process** do turns mein split ho sakta hai:
- Ek turn tool call decision ke liye (LLM se tool choose karna).
- Doosra turn tool result ko process aur final output banane ke liye.
Isliye total 3 turns ban rahe hain.

---

### **Solution**
Error fix karne ke liye, aapko `max_turns` ko badhana hoga taake sare zaroori turns complete ho saken. `max_turns=3` set karna chahiye kyunki aapka workflow 3 turns le raha hai.

#### **Updated Code**
```python
from dotenv import load_dotenv
import os
import asyncio
from pydantic import BaseModel
from agents import (
    Agent,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig,
    Runner,
    ModelSettings,
    function_tool,
)

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
base_url = os.getenv("BASE_URL")

external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/",
)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client,
)

config = RunConfig(
    model=model,
    model_provider=external_client,
    tracing_disabled=True,
)

@function_tool
def fetch_weather(city: str) -> str:
    return f"Weather of {city} is Cloudy and Temperature is 25C"

weather_agent = Agent(
    name="Weather_Agent",
    instructions="You are a Weather Assistant. Use the fetch_weather tool to answer weather-related queries.",
    model=model,
    tools=[fetch_weather],
)

agent = Agent(
    name="Assistant",
    instructions="You are a Powerful Assistant. Handoff weather-related queries to Weather_Agent.",
    model=model,
    handoffs=[weather_agent]
)

async def main():
    result = Runner.run_streamed(
        agent,
        "What's the weather in Gujar Khan",
        run_config=config,
        max_turns=3  # Increased to 3 to allow all turns
    )
    async for event in result.stream_events():
        print(f"Event: {event}")
        # Extract output and final_output for clarity
        if hasattr(event, "output") and event.output:
            print("Intermediate Output:", event.output)
            for output in event.output:
                if hasattr(output, "content"):
                    for content in output.content:
                        if hasattr(content, "text"):
                            print("Output Text:", content.text)
        if hasattr(event, "final_output") and event.final_output:
            print("Final Output:", event.final_output)

if __name__ == "__main__":
    asyncio.run(main())
```

**Changes**:
- `max_turns=3` set kiya taake main agent ka handoff (Turn 1), weather agent ka tool decision (Turn 2), aur tool result processing (Turn 3) complete ho saken.
- Event processing mein `output` aur `final_output` ko extract kiya taake streaming events ka flow clear ho.
- Logging add kiya (`print` statements) taake har turn ka output dekha ja sake.

**Expected Output**:
```
Event: <RunResultStreaming object with output=[ResponseOutputMessage(...text='Handoff to Weather_Agent...')]>
Intermediate Output: [ResponseOutputMessage(...text='Handoff to Weather_Agent...')]
Output Text: Handoff to Weather_Agent for weather query
Event: <RunResultStreaming object with output=[ResponseOutputMessage(...text='Calling fetch_weather tool...')]>
Intermediate Output: [ResponseOutputMessage(...text='Calling fetch_weather tool...')]
Output Text: Calling fetch_weather tool for Gujar Khan
Event: <RunResultStreaming object with final_output='Weather of Gujar Khan is Cloudy and Temperature is 25C'>
Intermediate Output: [ResponseOutputMessage(...text='Weather of Gujar Khan is Cloudy and Temperature is 25C')]
Output Text: Weather of Gujar Khan is Cloudy and Temperature is 25C
Final Output: Weather of Gujar Khan is Cloudy and Temperature is 25C
```

---

### **Alternative Solution: Optimize Turns**
Agar aap turns ko kam karna chahte hain taake `max_turns=2` mein kaam ho, to aapko workflow optimize karna hoga. Lekin aapke current setup mein (handoff + tool use), 3 turns zaroori hain kyunki:
- Handoff ek turn leta hai.
- Tool decision ek turn leta hai.
- Tool result processing ek turn leta hai.

Agar aap handoff ke bajaye **single agent** use karen (weather agent directly query le), to shayad turns kam ho saken (2 turns: tool decision + result processing). Example:

```python
async def main():
    result = Runner.run_streamed(
        weather_agent,  # Directly use Weather_Agent
        "What's the weather in Gujar Khan",
        run_config=config,
        max_turns=2
    )
    async for event in result.stream_events():
        print(f"Event: {event}")
        if hasattr(event, "output") and event.output:
            print("Intermediate Output:", event.output)
            for output in event.output:
                if hasattr(output, "content"):
                    for content in output.content:
                        if hasattr(content, "text"):
                            print("Output Text:", content.text)
        if hasattr(event, "final_output") and event.final_output:
            print("Final Output:", event.final_output)
```

**Kyun Kaam Karega?**:
- Handoff ka turn khatam ho jayega kyunki `weather_agent` directly query process karega.
- Total turns:
  - Turn 1: Tool decision (LLM call for `fetch_weather`).
  - Turn 2: Tool result processing aur final output.
- `max_turns=2` sufficient hoga.

**Expected Output**:
```
Event: <RunResultStreaming object with output=[ResponseOutputMessage(...text='Calling fetch_weather tool...')]>
Intermediate Output: [ResponseOutputMessage(...text='Calling fetch_weather tool...')]
Output Text: Calling fetch_weather tool for Gujar Khan
Event: <RunResultStreaming object with final_output='Weather of Gujar Khan is Cloudy and Temperature is 25C'>
Intermediate Output: [ResponseOutputMessage(...text='Weather of Gujar Khan is Cloudy and Temperature is 25C')]
Output Text: Weather of Gujar Khan is Cloudy and Temperature is 25C
Final Output: Weather of Gujar Khan is Cloudy and Temperature is 25C
```

---

### **Kyun Error Aaya?**
- **Main Reason**: Aapka workflow 3 turns le raha hai (handoff, tool decision, tool result processing), lekin `max_turns=2` set tha, isliye 3rd turn pe `MaxTurnsExceeded` error aaya.
- **Aapka Confusion**: Aapko laga ke handoff aur tool use 2 turns mein complete ho jayega, lekin tool use ka process 2 turns (decision + result processing) mein split ho raha hai.
- **SDK Behavior**: OpenAI-Agents SDK mein har logical step (handoff, tool call decision, result processing) ek alag turn count hota hai.

---

### **Additional Checks**
1. **SDK Version**:
   - Ensure karen ke aap latest OpenAI-Agents SDK use kar rahe hain:
     ```bash
     pip show openai-agents
     ```
     Update karen:
     ```bash
     pip install --upgrade openai-agents
     ```

2. **Logging**:
   - Debugging ke liye logging enable karen:
     ```python
     import logging
     logging.basicConfig(level=logging.DEBUG)
     ```
   - Yeh har turn, handoff, aur tool call ka detail dikhayega.

3. **Base URL Aur API Key**:
   - `.env` file check karen:
     ```
     GEMINI_API_KEY=your_valid_key
     BASE_URL=https://generativelanguage.googleapis.com/v1beta/
     ```
   - Galat API key ya base_url connection issues de sakta hai.

4. **Event Structure**:
   - Events ke structure ko confirm karen:
     ```python
     async for event in result.stream_events():
         print(dir(event))  # Event ke available properties dekhein
     ```

---

### **Asaan Summary**
- **Problem**: `MaxTurnsExceeded` error isliye aaya kyunki agent 3 turns le raha hai (handoff, tool decision, tool result processing), lekin `max_turns=2` set tha.
- **Kitne Turns?**:
  - Turn 1: Main agent ka handoff.
  - Turn 2: Weather agent ka tool decision.
  - Turn 3: Weather agent ka tool result processing.
- **Solution**:
  - `max_turns=3` set karen taake sare turns complete ho.
  - Alternative: Handoff skip karen aur `weather_agent` directly use karen, jo 2 turns mein kaam karega.
- **Kyun Hua**: Tool use ka process 2 turns mein split hota hai, aur handoff ek extra turn add karta hai.

Agar aapko aur koi error aaye ya specific part samajhna hai (jaise turns ka flow ya event structure), to error message ya details share karen! Main aur help karunga.

---------
### This Code Tree Structure:

```python
[RunResultStreaming(
    input="What's the weather in Gujar Khan",
    new_items=[],
    raw_responses=[],
    final_output=None,
    input_guardrail_results=[],
    output_guardrail_results=[],
    context_wrapper=RunContextWrapper(
        context=None,
        usage=Usage(
            requests=0,
            input_tokens=0,
            input_tokens_details=InputTokensDetails(cached_tokens=0),
            output_tokens=0,
            output_tokens_details=OutputTokensDetails(reasoning_tokens=0),
            total_tokens=0
        )
    ),
    current_agent=Agent(
        name='Assistant',
        instructions='You are a Powerful Assistant. Handoff weather-related queries to Weather_Agent.',
        handoff_description=None,
        handoffs=[
            Agent(
                name='Weather_Agent',
                instructions='You are a Weather Assistant. Use the fetch_weather tool to answer weather-related queries.',
                handoff_description=None,
                handoffs=[],
                model=OpenAIChatCompletionsModel(...),
                model_settings=ModelSettings(
                    temperature=None,
                    top_p=None,
                    frequency_penalty=None,
                    presence_penalty=None,
                    tool_choice=None,
                    parallel_tool_calls=None,
                    truncation=None,
                    max_tokens=None,
                    reasoning=None,
                    metadata=None,
                    store=None,
                    include_usage=None,
                    extra_query=None,
                    extra_body=None,
                    extra_headers=None
                ),
                tools=[
                    FunctionTool(
                        name='fetch_weather',
                        description='',
                        params_json_schema={
                            'properties': {
                                'city': {'title': 'City', 'type': 'string'}
                            },
                            'required': ['city'],
                            'title': 'fetch_weather_args',
                            'type': 'object',
                            'additionalProperties': False
                        },
                        on_invoke_tool=<function ...>,
                        strict_json_schema=True
                    )
                ],
                mcp_servers=[],
                mcp_config={},
                input_guardrails=[],
                output_guardrails=[],
                output_type=None,
                hooks=None,
                tool_use_behavior='run_llm_again',
                reset_tool_choice=True
            )
        ],
        model=OpenAIChatCompletionsModel(...),
        model_settings=ModelSettings(
            temperature=None,
            top_p=None,
            frequency_penalty=None,
            presence_penalty=None,
            tool_choice=None,
            parallel_tool_calls=None,
            truncation=None,
            max_tokens=None,
            reasoning=None,
            metadata=None,
            store=None,
            include_usage=None,
            extra_query=None,
            extra_body=None,
            extra_headers=None
        ),
        tools=[],
        mcp_servers=[],
        mcp_config={},
        input_guardrails=[],
        output_guardrails=[],
        output_type=None,
        hooks=None,
        tool_use_behavior='run_llm_again',
        reset_tool_choice=True
    ),
    current_turn=0,
    max_turns=1,
    is_complete=False
)]
```

## **7.Understanding events in streaming:**

In [9]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=model
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",

    )

    async for event in result.stream_events():
      print(event)


asyncio.run(main())

AgentUpdatedStreamEvent(new_agent=Agent(name='Joker', instructions='First call the `how_many_jokes` tool, then tell that many jokes.', prompt=None, handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x798805444b90>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[FunctionTool(name='how_many_jokes', description='', params_json_schema={'properties': {}, 'title': 'how_many_jokes_args', 'type': 'object', 'additionalProperties': False, 'required': []}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7988062ec2c0>, strict_json_schema=True, is_enabled=True)], mcp_servers=[], mcp_config={}, input_guar


---

### **1. `AgentUpdatedStreamEvent`**
- **Class**: `AgentUpdatedStreamEvent`
- **Details**: Yeh event ek naye agent ko introduce karta hai, jiska naam 'Joker' hai aur instructions hain "You are a helpful assistant." Ismein model (OpenAIChatCompletionsModel) aur settings bhi shamil hain.
- **Kaam**: Yeh batata hai ke kaunsa agent aapki query ko handle karega aur uski configuration kya hai.
- **Purpose**: Process ka starting point set karta hai, taaki system ko pata chale ke kaunsa assistant kaam karega.
- **Asaan Alfaaz Mein**: "System ne 'Joker' naam ka assistant chuna hai jo aapki madad karega. Yeh ab kaam shuru karne ke liye tayyar hai."

---

### **2. `RawResponsesStreamEvent` (ResponseCreatedEvent)**
- **Class**: `RawResponsesStreamEvent` jismein `ResponseCreatedEvent` hai.
- **Details**: Ismein ek naya response shuru hota hai, jismein model 'gemini-2.0-flash' use ho raha hai, aur basic details jaise ID aur timestamp hain.
- **Kaam**: Yeh ek naye response ki shuruaat ko mark karta hai.
- **Purpose**: System ko batata hai ke response generate karna shuru ho gaya hai.
- **Asaan Alfaaz Mein**: "System ne aapke sawal ka jawab banana shuru kar diya hai, aur yeh 'gemini-2.0-flash' model se banega."

---

### **3. `RawResponsesStreamEvent` (ResponseOutputItemAddedEvent)**
- **Class**: `RawResponsesStreamEvent` jismein `ResponseOutputItemAddedEvent` hai.
- **Details**: Ek naya output item (message) add hota hai jo assistant se aayega, aur yeh abhi "in_progress" mein hai.
- **Kaam**: Response mein assistant ka message shuru karta hai.
- **Purpose**: Yeh batata hai ke message ban raha hai, lekin abhi pura nahi hua.
- **Asaan Alfaaz Mein**: "Assistant ne aapke liye message likhna shuru kiya hai, lekin abhi woh adhoora hai."

---

### **4. `RawResponsesStreamEvent` (ResponseContentPartAddedEvent)**
- **Class**: `RawResponsesStreamEvent` jismein `ResponseContentPartAddedEvent` hai.
- **Details**: Message mein ek content part add hota hai, jo ek text part hai, lekin abhi khali hai.
- **Kaam**: Message ke ek section ko shuru karta hai jismein text aayega.
- **Purpose**: Yeh text ka placeholder banata hai jo aage bhara jayega.
- **Asaan Alfaaz Mein**: "Message ka ek hissa banaya gaya hai jismein text aayega, lekin abhi woh khali hai."

---

### **5. `RawResponsesStreamEvent` (ResponseTextDeltaEvent)** *(Multiple Instances)*
- **Class**: `RawResponsesStreamEvent` jismein `ResponseTextDeltaEvent` hai.
- **Details**: Yeh event kai baar aata hai, har baar message mein thoda text add karta hai, jaise:
  - "Alright"
  - ", here are 5 jokes for you:\n\n1.  Why don'"
  - "t scientists trust atoms?\n    Because they make up everything!\n\n2."
  - Aur aage bhi.
- **Kaam**: Message ka text chhote-chhote hisson mein incrementally add hota hai.
- **Purpose**: Real-time mein response ko build karta hai, taaki aapko jawab banta hua dikhe.
- **Asaan Alfaaz Mein**: "Assistant message ko thoda-thoda likh raha hai, jaise pehle 'Alright', phir 'here are 5 jokes', aur dheere-dheere pura text ban raha hai."

---

### **6. `RawResponsesStreamEvent` (ResponseContentPartDoneEvent)**
- **Class**: `RawResponsesStreamEvent` jismein `ResponseContentPartDoneEvent` hai.
- **Details**: Yeh event batata hai ke message ka text part pura ho gaya hai, aur final text hai: "Alright, here are 5 jokes for you..." (5 jokes ke saath).
- **Kaam**: Message ke ek content part ko complete karta hai.
- **Purpose**: Yeh confirm karta hai ke text ka yeh hissa ab final hai.
- **Asaan Alfaaz Mein**: "Message ka text hissa pura ho gaya hai, aur ab yeh 5 jokes ke saath ready hai."

---

### **7. `RawResponsesStreamEvent` (ResponseOutputItemDoneEvent)**
- **Class**: `RawResponsesStreamEvent` jismein `ResponseOutputItemDoneEvent` hai.
- **Details**: Assistant ka pura message complete ho gaya hai (status: "completed"), aur yeh 5 jokes wala text hai.
- **Kaam**: Response ka output item (message) complete karta hai.
- **Purpose**: Yeh batata hai ke assistant ka message ab final aur tayyar hai.
- **Asaan Alfaaz Mein**: "Assistant ka pura message ban gaya hai, aur ab yeh aapko dikhane ke liye ready hai."

---

### **8. `RawResponsesStreamEvent` (ResponseCompletedEvent)**
- **Class**: `RawResponsesStreamEvent` jismein `ResponseCompletedEvent` hai.
- **Details**: Pura response complete ho gaya hai, jismein final message (5 jokes) aur details jaise model aur ID shamil hain.
- **Kaam**: Pura response process khatam karta hai.
- **Purpose**: Yeh confirm karta hai ke jawab final hai aur aapko deliver ho sakta hai.
- **Asaan Alfaaz Mein**: "Aapka pura jawab tayyar ho gaya hai, aur ab system ne ise complete mark kar diya hai."

---

## **9. `RunItemStreamEvent`:**

### **Saare Events Ka Chhota Overview**
Yeh events do tarah ke hain:
1. **`RawResponsesStreamEvent`**: Yeh chhote-chhote steps ko track karte hain, jaise response shuru hona, tool call add hona, ya text ka ek hissa likha jana. Yeh zyada technical aur detailed hote hain.
2. **`RunItemStreamEvent`**: Yeh bade aur important steps ko dikhate hain, jaise tool call hona, tool ka result milna, ya final message ban jana. Yeh user-friendly hote hain aur agent ke kaam ka flow samajhne mein madad karte hain.

Aapke data mein teeno `RunItemStreamEvent` hain, aur main inko aapke liye explain karunga. Pehle yeh samajh lete hain ke pura process kya hai, phir `RunItemStreamEvent` par gaur karenge.

---

### **`RunItemStreamEvent` Kya Hai?**
Ab specifically `RunItemStreamEvent` ko samajhte hain. Yeh event agent ke bade actions ko track karta hai, taaki aapko pata chale ke assistant ne kya kiya aur kya result mila. Yeh teen baar aata hai aapke data mein, aur har baar ka alag matlab hai:

#### **1. `RunItemStreamEvent` (name='tool_called')**
- **Details**: Yeh batata hai ke 'Joker' agent ne `how_many_jokes` tool ko call kiya hai.
- **Kaam**: Tool ko chalane ka action shuru karna.
- **Kyun Zaroori Hai**: Yeh dikhata hai ke agent ne tool use karne ka faisla liya aur ab tool kaam karega.
- **Asaan Alfaaz Mein**: "Assistant ne bola, 'Chalo, `how_many_jokes` tool chalate hain!'"

#### **2. `RunItemStreamEvent` (name='tool_output')**
- **Details**: Yeh batata hai ke `how_many_jokes` tool ne apna result diya hai, jo '5' hai (yani 5 jokes batane hain).
- **Kaam**: Tool ka output agent ko dena, taaki woh aage kaam kar sake.
- **Kyun Zaroori Hai**: Yeh agent ko information deta hai ke kitne jokes batane hain.
- **Asaan Alfaaz Mein**: "Tool ne jawab diya, '5 jokes chahiye.'"

#### **3. `RunItemStreamEvent` (name='message_output_created')**
- **Details**: Yeh batata hai ke 'Joker' agent ne final message bana diya hai, jismein 5 jokes hain:
  - "I am programmed to tell 5 jokes.  
    1. Why don't scientists trust atoms? Because they make up everything!  
    2. Parallel lines have so much in common. It’s a shame they’ll never meet.  
    3. Why did the scarecrow win an award? Because he was outstanding in his field!  
    4. I told my wife she was drawing her eyebrows too high. She seemed surprised.  
    5. What do you call a fish with no eyes? Fsh!"
- **Kaam**: Pura message conversation mein add karna.
- **Kyun Zaroori Hai**: Yeh confirm karta hai ke agent ka kaam khatam ho gaya aur jawab aapko mil gaya.
- **Asaan Alfaaz Mein**: "Assistant ne 5 jokes likhkar aapko message de diya."


---

### **Summary**
Yeh saare events ek tarteeb se kaam karte hain:
1. **Agent chuna jata hai** (`AgentUpdatedStreamEvent`).
2. **Response shuru hota hai** (`ResponseCreatedEvent`).
3. **Message banaya jata hai** (`ResponseOutputItemAddedEvent`).
4. **Text thoda-thoda add hota hai** (`ResponseTextDeltaEvent`).
5. **Text part pura hota hai** (`ResponseContentPartDoneEvent`).
6. **Message pura hota hai** (`ResponseOutputItemDoneEvent`).
7. **Response complete hota hai** (`ResponseCompletedEvent`).
8. **Message conversation mein jata hai** (`RunItemStreamEvent`).

Yeh process real-time mein hota hai, jisse aap dekh sakte hain ke assistant kaise jawab bana raha hai—jaise chat mein "typing..." dikhayi deta hai. Har event ka apna kaam hota hai, aur saath mein yeh ek smooth streaming response banate hain. Agar aapko kisi event ke baare mein aur detail chahiye, to zaroor batayein!

In [13]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=model
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",

    )
    print("=== Run starting ===")
    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                text = event.item.raw_item.content[0].text
                print(f"-- Message output:\n {text}")
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types



asyncio.run(main())

print("=== Run complete ===")


=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 7
-- Message output:
 I am programmed to tell 7 jokes.

1. Why don't scientists trust atoms? Because they make up everything!
2. Parallel lines have so much in common. It’s a shame they’ll never meet.
3. Why did the scarecrow win an award? Because he was outstanding in his field!
4. I told my wife she was drawing her eyebrows too high. She seemed surprised.
5. What do you call a lazy kangaroo? Pouch potato!
6. Why did the bicycle fall over? Because it was two tired!
7. I'm reading a book on anti-gravity. It's impossible to put down!

=== Run complete ===


# **8: Understanding: {ItemHelpers.text_message_output(event.item)}**

----
Aapka sawal yeh hai ke `RunItemStreamEvent` mein text kahan hai jise aap access kar rahe hain aur `ItemHelpers.text_message_output` helper ka kya kaam hai. Main ise aapko step-by-step samjhaata hoon taaki sab clear ho jaye.

---

### **Text Kahan Hai?**
Aapke diya hua event ek `RunItemStreamEvent` hai, aur iska structure kuch aisa hai:

```python
RunItemStreamEvent(
    name='message_output_created',
    item=MessageOutputItem(
        agent=Agent(...),  # Agent ki details
        raw_item=ResponseOutputMessage(
            id='__fake_id__',
            content=[ResponseOutputText(
                annotations=[],
                text="I am programmed to tell 5 jokes.\n\n1. Why don't scientists trust atoms? Because they make up everything!\n2. Parallel lines have so much in common. It’s a shame they’ll never meet.\n3. Why did the scarecrow win an award? Because he was outstanding in his field!\n4. I told my wife she was drawing her eyebrows too high. She seemed surprised.\n5. What do you call a fish with no eyes? Fsh!\n",
                type='output_text',
                logprobs=None
            )],
            role='assistant',
            status='completed',
            type='message'
        ),
        type='message_output_item'
    ),
    type='run_item_stream_event'
)
```

Is event mein text nikalne ke liye humein structure ke andar jana hoga:

1. **`event.item`**: Yeh ek `MessageOutputItem` object hai.
2. **`event.item.raw_item`**: Ismein `ResponseOutputMessage` object hai, jo asli message ko hold karta hai.
3. **`event.item.raw_item.content`**: Yeh ek list hai jismein `ResponseOutputText` objects hote hain. Is case mein sirf ek object hai.
4. **`event.item.raw_item.content[0].text`**: Yahan actual text hai jo aap chahte hain.

**Text ki Location**:  
Text `event.item.raw_item.content[0].text` mein milta hai. Yani agar aap ise access karna chahein, to aap aise kar sakte hain:

```python
text = event.item.raw_item.content[0].text
print(text)
# Output: "I am programmed to tell 5 jokes.\n\n1. Why don't scientists trust atoms? ..."
```

Yeh wohi 5 jokes wala message hai jo aapke event mein hai.

---

### **Helper (`ItemHelpers.text_message_output`) Kya Hai?**
Ab baat karte hain aapke code ki is line ki:

```python
elif event.item.type == "message_output_item":
    print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
```

#### **Helper Ka Kaam**
- **`ItemHelpers.text_message_output(event.item)`**: Yeh ek function hai jo `MessageOutputItem` object se text ko extract karta hai aur aapko seedha deta hai.
- **Kaise Kaam Karta Hai?**: Yeh shayad `event.item.raw_item.content[0].text` ko return karta hai, ya agar `content` mein multiple text objects hote hain to unko combine karke deta hai.
- **Purpose**: Iska maqsad yeh hai ke aapko event ke complex structure mein manually jane ki zarurat na pade. Yeh helper aapka kaam asaan karta hai.

**Asaan Alfaaz Mein**:  
Helper aapko woh text seedha haath mein de deta hai jo 5 jokes ke saath hai, bina yeh sochne ke ke "text kahan hai, kaise nikalu?". For example, agar aap manually `event.item.raw_item.content[0].text` likhte, to bhi kaam ho jata, lekin helper ke saath code chhota aur saaf rehta hai.

#### **Helper Ke Fayde**
1. **Simplification**: Aapko structure ke andar dhundhne ki zarurat nahi.
2. **Readability**: Code zyada clear aur samajhne mein aasan hota hai.
3. **Maintainability**: Agar event ka structure future mein badalta hai, to helper us hisaab se adjust ho sakta hai.

**Example**:
- **Bina Helper Ke**:
  ```python
  text = event.item.raw_item.content[0].text
  print(f"-- Message output:\n {text}")
  ```
- **Helper Ke Saath**:
  ```python
  print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
  ```
  Dono ka output same hoga, lekin helper wala tariqa zyada clean aur professional hai.

---

### **Summary**
- **Text Kahan Hai?**: `event.item.raw_item.content[0].text` mein. Yeh woh jagah hai jahan 5 jokes ka message store hai.
- **Helper Kya Hai?**: `ItemHelpers.text_message_output(event.item)` ek function hai jo text ko event se nikaal ke deta hai, taaki aapko manually structure explore na karna pade.
- **Fayda**: Yeh aapke code ko simple, readable, aur future-proof banata hai.

-----------

## **9. Difference B/W FunctionTool and function_tool:**

Aapka sawal OpenAI-Agents SDK mein `FunctionTool` aur `function_tool` ke beech ke farq se related hai, jahan aap dono ko import kar rahe hain:

```python
from agents import (
    function_tool,
    FunctionTool
)
```

Main aapko asaan zabaan mein, step-by-step samjhaunga ke yeh dono kya hain, unmein kya farq hai, aur kaise kaam karte hain.

---

### **1. `FunctionTool` Kya Hai?**
- **Kya Hai?**: `FunctionTool` ek **class** hai jo OpenAI-Agents SDK mein define ki gayi hai. Yeh ek tool ko represent karta hai jise agent use kar sakta hai, jaise koi function jo specific kaam karta ho (misal: `fetch_weather`).
- **Kaam**: Yeh class ek structured way provide karta hai jismein aap ek tool define karte hain, uske parameters, JSON schema, aur behavior set karte hain.
- **Structure**: Ismein fields hote hain jaise:
  - `name`: Tool ka naam.
  - `description`: Tool kya karta hai.
  - `params_json_schema`: Tool ke input parameters ka JSON schema.
  - `on_invoke_tool`: Ek callable function jo tool chalane pe run hota hai.
  - `strict_json_schema`: Schema validation ke liye boolean.
- **Kab Use Hota Hai?**: Jab aap manually ek tool object banate hain ya advanced customization chahiye.

**Example**:
```python
from agents import FunctionTool

def my_function(city: str) -> str:
    return f"Weather in {city} is sunny!"

weather_tool = FunctionTool(
    name="fetch_weather",
    description="Fetches weather for a city",
    params_json_schema={
        "properties": {"city": {"type": "string", "title": "City"}},
        "required": ["city"],
        "type": "object"
    },
    on_invoke_tool=my_function,
    strict_json_schema=True
)
```

**Asaan Alfaaz Mein**: `FunctionTool` ek blueprint hai jisse aap ek tool banate hain, jismein aap sab kuch manually set karte hain—naam, description, parameters, aur function jo chalega.

---

### **2. `function_tool` Kya Hai?**
- **Kya Hai?**: `function_tool` ek **decorator** function hai jo OpenAI-Agents SDK mein diya gaya hai. Yeh ek Python function ko `FunctionTool` object mein convert karta hai.
- **Kaam**: Yeh aapke function ke liye automatically `FunctionTool` banata hai, jismein function ke signature (parameters aur return type) se JSON schema generate hota hai, aur function khud `on_invoke_tool` ke roop mein set hota hai.
- **Structure**: Yeh decorator aapke function ko wrap karta hai aur ek `FunctionTool` instance return karta hai. Ismein bhi wahi fields hote hain jo `FunctionTool` class mein hote hain, lekin zyadatar cheezein automatically set ho jati hain.
- **Kab Use Hota Hai?**: Jab aap ek simple function ko jaldi se tool mein badalna chahte hain bina manually schema ya settings likhe.

**Example**:
```python
from agents import function_tool

@function_tool
def fetch_weather(city: str) -> str:
    return f"Weather of {city} is Cloudy and Temperature is 25C"
```

- **Kya Hota Hai?**: Yeh decorator `fetch_weather` function ko ek `FunctionTool` object mein convert karta hai. Automatically:
  - `name="fetch_weather"` set hota hai.
  - `params_json_schema` function ke parameter (`city: str`) se ban jata hai.
  - `on_invoke_tool` ko `fetch_weather` function assign hota hai.
  - Default settings (jaise `strict_json_schema=True`) apply hoti hain.

**Asaan Alfaaz Mein**: `function_tool` ek shortcut hai jo aapke function ko ek tool mein badal deta hai, aur aapko zyada likhne ki zarurat nahi padti.

---

### **Kaise Decide Karen Kya Use Karna Hai?**
- **`FunctionTool` Use Karen Jab**:
  - Aapko ek tool ke liye specific JSON schema chahiye jo function ke signature se alag ho.
  - Aap non-Python function (jaise external API call) ko tool banane ke liye use karna chahte hain.
  - Aapko description ya extra settings (jaise custom validation) add karne hain.
  - Example: Agar aap ek API call ko tool banate hain jiska schema function se nahi ban sakta.

- **`function_tool` Use Karen Jab**:
  - Aapka tool ek simple Python function hai jiska schema function ke parameters se ban sakta hai.
  - Aap jaldi aur kam code mein tool banana chahte hain.
  - Example: Aapke code mein `fetch_weather` jaisa function jo direct result deta hai.

---

### **Aapke Code Mein Context**
Aapke pichle code mein aapne `function_tool` ka use kiya tha:

```python
@function_tool
def fetch_weather(city: str) -> str:
    return f"Weather of {city} is Cloudy and Temperature is 25C"
```

Yahan `function_tool` decorator ne `fetch_weather` ko ek `FunctionTool` object mein convert kiya, jo `Weather_Agent` ke `tools` list mein add hua. Isne automatically:
- `name="fetch_weather"` set kiya.
- `params_json_schema` banaya (`city` parameter ke liye).
- `on_invoke_tool` ko `fetch_weather` function assign kiya.

Agar aap yeh manually `FunctionTool` se karte, to zyada code likhna padta:

```python
weather_tool = FunctionTool(
    name="fetch_weather",
    params_json_schema={
        "properties": {"city": {"type": "string", "title": "City"}},
        "required": ["city"],
        "type": "object",
        "additionalProperties": False
    },
    on_invoke_tool=fetch_weather,
    strict_json_schema=True
)
```

---

### **Technical Details**
- **Internal Working**:
  - `function_tool` decorator shayad `FunctionTool` class ka ek instance banata hai. Iska source code shayad kuch aisa hoga (pseudocode):
    ```python
    def function_tool(func):
        schema = generate_schema_from_function(func)
        return FunctionTool(
            name=func.__name__,
            params_json_schema=schema,
            on_invoke_tool=func,
            strict_json_schema=True
        )
    ```
  - Yeh Python ke function signature (type hints jaise `city: str`) se JSON schema banata hai.

- **Source Code Reference**:
  - OpenAI-Agents SDK ke GitHub repo (https://github.com/openai/openai-agents-python) mein `agents/tools.py` ya similar file mein `FunctionTool` class aur `function_tool` decorator ka implementation mil sakta hai.
  - `FunctionTool` ek base class hai jo shayad `Tool` class se inherit karta hai, aur `function_tool` iska wrapper hai.


